In [ ]:
import rasterio
from rasterio.enums import Resampling

in_image = r"C:\BERATools\Surmont_New_AOI\Merged_CHM_2022.tif"
out_image = r"C:\BERATools\Surmont_New_AOI\1m_down_sample\Merged_CHM_2022.tif"

scale_factor = 0.3

with rasterio.open(in_image) as dataset:

    # resample data to target shape using scale_factor
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * scale_factor),
            int(dataset.width * scale_factor)
        ),
        resampling=Resampling.bilinear
    )

    dst_crs = dataset.crs

    print('Shape before resample:', dataset.shape)
    print('Shape after resample:', data.shape[1:])

    # scale image transform
    dst_transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

    print('Transform before resample:\n', dataset.transform, '\n')
    print('Transform after resample:\n', dst_transform)

    ## Write outputs
    # set properties for output
    dst_kwargs = dataset.meta.copy()
    dst_kwargs.update(
        {
            "crs": dst_crs,
            "transform": dst_transform,
            "width": data.shape[-1],
            "height": data.shape[-2],
            "nodata": 0,  
        }
    )

    with rasterio.open(out_image, "w", **dst_kwargs) as dst:
        # iterate through bands
        for i in range(data.shape[0]):
              dst.write(data[i].astype(rasterio.uint32), i+1)